## Imports

In [1]:
import pandas as pd
import numpy as np
import re

## Import Data

In [2]:
titles_imdb = pd.read_csv('../data/title.basics.tsv', sep='\t')
ratings_imdb = pd.read_csv('../data/title.ratings.tsv', sep='\t')
budgets_kaggle = pd.read_csv('../data/kaggle_movies_metadata.csv')

/var/folders/6k/g222zmkd5n9702hv3gdmx26h0000gn/T/ipykernel_3440/1476460092.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  titles_imdb = pd.read_csv('../data/title.basics.tsv', sep='\t')
/var/folders/6k/g222zmkd5n9702hv3gdmx26h0000gn/T/ipykernel_3440/1476460092.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  budgets_kaggle = pd.read_csv('../data/kaggle_movies_metadata.csv')


In [3]:
titles_imdb.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [4]:
ratings_imdb.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1922
1,tt0000002,5.8,259
2,tt0000003,6.5,1734
3,tt0000004,5.6,174
4,tt0000005,6.2,2545


In [5]:
budgets_kaggle.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [6]:
# Importing CMU Datasets
movie = pd.read_csv('./../data/MovieSummaries/movie.metadata.tsv', 
                 sep='\t',
                 names=['Wikipedia_movie_ID',
                        'Freebase_movie_ID',
                        'Movie_name',
                        'Movie_release_date',
                        'Movie_box_office_revenue',
                        'Movie_runtime',
                        'Movie_languages',
                        'Movie_countries',
                        'Movie_genres'],
                 header=None)

character = pd.read_csv('./../data/MovieSummaries/character.metadata.tsv', 
                 sep='\t',
                 names=['Wikipedia_movie_ID',
                        'Freebase_movie_ID',
                        'Movie_release_date',
                        'Character_name',
                        'Actor_date_of_birth',
                        'Actor_gender',
                        'Actor_height',
                        'Actor_ethnicity',
                        'Actor_name',
                        'Actor_age_at_movie_release',
                        'Freebase_character/actor_map_ID',
                        'Freebase_character_ID',
                        'Freebase_actor_ID'],
                 header=None)

In [7]:
movie.head()

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


In [8]:
character.head()

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_release_date,Character_name,Actor_date_of_birth,Actor_gender,Actor_height,Actor_ethnicity,Actor_name,Actor_age_at_movie_release,Freebase_character/actor_map_ID,Freebase_character_ID,Freebase_actor_ID
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg


# Data cleaning

### Date change

In [9]:
#move release dates and actor birth dates to datetime format and keep only release year
character.Movie_release_date = pd.to_datetime(character['Movie_release_date'],
                                              errors='coerce').dt.year

movie.Movie_release_date = pd.to_datetime(movie['Movie_release_date'],
                                              errors='coerce').dt.year

In [10]:
#sort dataframes by ascending release year
character.sort_values(by=['Movie_release_date'], ascending=True, inplace=True)

movie.sort_values(by=['Movie_release_date'], ascending=True, inplace=True)

### Null value removal

In [11]:
# Nan percentage for character dataset
character.isnull().sum() * 100 / character.shape[0]

Wikipedia_movie_ID                  0.000000
Freebase_movie_ID                   0.000000
Movie_release_date                  2.221142
Character_name                     57.220488
Actor_date_of_birth                23.552763
Actor_gender                       10.120288
Actor_height                       65.645740
Actor_ethnicity                    76.466542
Actor_name                          0.272484
Actor_age_at_movie_release         35.084064
Freebase_character/actor_map_ID     0.000000
Freebase_character_ID              57.218269
Freebase_actor_ID                   0.180842
dtype: float64

In [12]:
character = character[['Wikipedia_movie_ID', 'Actor_name', 'Actor_gender', 'Actor_date_of_birth']]
character

,Wikipedia_movie_ID,Actor_name,Actor_gender,Actor_date_of_birth
123565,1082508,Harriet Hartley,F,NaN
123564,1082508,Joseph Whitley,M,NaN
123563,1082508,Sarah Whitley,F,NaN
123562,1082508,Adolphe Le Prince,M,NaN
72684,644824,William Dickson,M,1860-08-03
...,...,...,...,...
450334,29119097,Biswajit Chakraborty,M,NaN
450494,17198681,Melissa Errico,F,1970-03-23
450495,17198681,Charlie Sheen,M,1965-09-03
450496,17198681,Renée Estevez,F,1967-04-02


In [13]:
char_no_nan = character.dropna(axis=0, how='any')
char_no_nan

,Wikipedia_movie_ID,Actor_name,Actor_gender,Actor_date_of_birth
72684,644824,William Dickson,M,1860-08-03
294940,73372,John Ott,M,1909-10-23
342275,786716,William Dickson,M,1860-08-03
6955,10104497,James J. Corbett,M,1866-09-01
229796,1090697,Fred Ott,M,1860
...,...,...,...,...
450333,29119097,Deepankar De,M,1944-07-05
450494,17198681,Melissa Errico,F,1970-03-23
450495,17198681,Charlie Sheen,M,1965-09-03
450496,17198681,Renée Estevez,F,1967-04-02


In [14]:
# Nan percentage for movie dataset
movie.isnull().sum() * 100 / movie.shape[0]

Wikipedia_movie_ID           0.000000
Freebase_movie_ID            0.000000
Movie_name                   0.000000
Movie_release_date           8.444966
Movie_box_office_revenue    89.722416
Movie_runtime               25.018045
Movie_languages              0.000000
Movie_countries              0.000000
Movie_genres                 0.000000
dtype: float64

In [15]:
movie = movie[['Wikipedia_movie_ID', 'Movie_name', 'Movie_release_date', 'Movie_box_office_revenue','Movie_runtime', 'Movie_countries', 'Movie_genres']]
movie

,Wikipedia_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_countries,Movie_genres
57963,1082508,Roundhay Garden Scene,1888.0,NaN,0.03,"{""/m/07ssc"": ""United Kingdom""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen..."
38764,12170539,"Leisurely Pedestrians, Open Topped Buses and H...",1889.0,NaN,NaN,"{""/m/07ssc"": ""United Kingdom""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen..."
60249,22770416,London's Trafalgar Square,1890.0,NaN,NaN,"{""/m/07ssc"": ""United Kingdom""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen..."
53907,6431996,"Monkeyshines, No. 1",1890.0,NaN,NaN,"{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film""}"
17849,27773121,Newark Athlete,1891.0,NaN,0.20,{},"{""/m/02hmvc"": ""Short Film""}"
...,...,...,...,...,...,...,...
81690,35758766,Love Torn in Dream,NaN,NaN,123.00,"{""/m/0f8l9c"": ""France"", ""/m/01p1v"": ""Chile"", ""...","{""/m/01hmnh"": ""Fantasy"", ""/m/07s9rl0"": ""Drama""}"
81692,26219108,The Leopard Woman,NaN,NaN,70.00,"{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/02l7c8"": ""Roma..."
81711,31422084,Zone 39,NaN,NaN,95.00,{},"{""/m/06n90"": ""Science Fiction""}"
81715,29449643,Muthukku Muthaaga,NaN,NaN,147.00,{},"{""/m/05p553"": ""Comedy film""}"


In [16]:
movie_no_nan = movie.dropna(0, inplace= False, how='any')
movie_no_nan

/var/folders/6k/g222zmkd5n9702hv3gdmx26h0000gn/T/ipykernel_3440/2940552793.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  movie_no_nan = movie.dropna(0, inplace= False, how='any')


,Wikipedia_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_countries,Movie_genres
50991,1762869,The Squaw Man,1914.0,244700.0,74.0,"{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/0hfjk"": ""Weste..."
2634,3333,The Birth of a Nation,1915.0,50000000.0,190.0,"{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/0219x_"": ""Indi..."
24953,4499436,Way Down East,1920.0,4500000.0,147.0,"{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/0219x_"": ""Indi..."
32124,21135479,Little Lord Fauntleroy,1921.0,900000.0,112.0,"{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/07s9rl0"": ""Dra..."
40247,1346905,The Kid,1921.0,2500000.0,60.0,"{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/07s9rl0"": ""Dra..."
...,...,...,...,...,...,...,...
7934,31795894,Step Up 4ever,2012.0,138942405.0,97.0,"{""/m/09c7w0"": ""United States of America""}","{""/m/02l7c8"": ""Romance Film"", ""/m/07s9rl0"": ""D..."
38702,31462588,Moonrise Kingdom,2012.0,64466140.0,93.0,"{""/m/09c7w0"": ""United States of America""}","{""/m/05p553"": ""Comedy film"", ""/m/07s9rl0"": ""Dr..."
38876,31631257,American Reunion,2012.0,234736898.0,113.0,"{""/m/09c7w0"": ""United States of America""}","{""/m/02l7c8"": ""Romance Film"", ""/m/01z4y"": ""Com..."
78458,32556787,Gone,2012.0,18100189.0,95.0,"{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller""}"


In [17]:
budgets_kaggle = budgets_kaggle[['original_title', 'budget']]
budgets_kaggle

,original_title,budget
0,Toy Story,30000000
1,Jumanji,65000000
2,Grumpier Old Men,0
3,Waiting to Exhale,16000000
4,Father of the Bride Part II,0
...,...,...
45461,رگ خواب,0
45462,Siglo ng Pagluluwal,0
45463,Betrayal,0
45464,Satana likuyushchiy,0


In [18]:
budgets_kaggle['budget'] = budgets_kaggle['budget'].replace('0', np.nan)
budgets_kaggle.isna().sum()

original_title        0
budget            36573
dtype: int64

In [19]:
budget_no_nan = budgets_kaggle.dropna()

# Merging Datasets

In [20]:
# Merging movie titles dataset with ratings dataset
first_merge = titles_imdb.merge(ratings_imdb, how='inner')
first_merge.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",5.7,1922
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",5.8,259
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",6.5,1734
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short",5.6,174
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",6.2,2545


In [21]:
# Droping unnecessary columns
imdb = first_merge[['primaryTitle', 'averageRating', 'numVotes']]
imdb.head()

,primaryTitle,averageRating,numVotes
0,Carmencita,5.7,1922
1,Le clown et ses chiens,5.8,259
2,Pauvre Pierrot,6.5,1734
3,Un bon bock,5.6,174
4,Blacksmith Scene,6.2,2545


In [22]:
# Removing duplicate film titles and averaging the duplicate ratings
mean_rating_imdb = first_merge[['primaryTitle', 'averageRating']].groupby('primaryTitle').mean()
mean_rating_imdb

,averageRating
primaryTitle,
!Next?,5.20
!Que ve el Bisbe!,6.20
!Women Art Revolution,6.80
#,4.25
# My Ass,6.80
...,...
ärtico,5.60
él,6.10
êmîcêtôsêt: Many Bloodlines,8.30


In [23]:
# Removing duplicate film titles and adding together the duplicate votes
total_vote_imdb = first_merge[['primaryTitle', 'numVotes']].groupby('primaryTitle').sum()
total_vote_imdb

,numVotes
primaryTitle,
!Next?,18
!Que ve el Bisbe!,12
!Women Art Revolution,247
#,16
# My Ass,9
...,...
ärtico,120
él,19
êmîcêtôsêt: Many Bloodlines,14


In [24]:
# Merging the two above dataframes
no_dupe_imdb = mean_rating_imdb.merge(total_vote_imdb, 'inner', 'primaryTitle')
no_dupe_imdb

,averageRating,numVotes
primaryTitle,,
!Next?,5.20,18
!Que ve el Bisbe!,6.20,12
!Women Art Revolution,6.80,247
#,4.25,16
# My Ass,6.80,9
...,...,...
ärtico,5.60,120
él,6.10,19
êmîcêtôsêt: Many Bloodlines,8.30,14


In [25]:
# Merging the rating/votes dataframe with movie metadata with no nan values
rated_data = movie_no_nan.merge(no_dupe_imdb, 'inner', left_on='Movie_name', right_on='primaryTitle')
rated_data

,Wikipedia_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_countries,Movie_genres,averageRating,numVotes
0,1762869,The Squaw Man,1914.0,244700.0,74.0,"{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/0hfjk"": ""Weste...",5.833333,1418
1,3333,The Birth of a Nation,1915.0,50000000.0,190.0,"{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/0219x_"": ""Indi...",6.600000,46186
2,4499436,Way Down East,1920.0,4500000.0,147.0,"{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/0219x_"": ""Indi...",6.750000,5791
3,21135479,Little Lord Fauntleroy,1921.0,900000.0,112.0,"{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/07s9rl0"": ""Dra...",6.857143,7444
4,1346905,The Kid,1921.0,2500000.0,60.0,"{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/07s9rl0"": ""Dra...",7.425000,182176
...,...,...,...,...,...,...,...,...,...
7690,32683873,The Odd Life of Timothy Green,2012.0,50466747.0,104.0,"{""/m/09c7w0"": ""United States of America""}","{""/m/01hmnh"": ""Fantasy"", ""/m/07s9rl0"": ""Drama""...",7.533333,47367
7691,31462588,Moonrise Kingdom,2012.0,64466140.0,93.0,"{""/m/09c7w0"": ""United States of America""}","{""/m/05p553"": ""Comedy film"", ""/m/07s9rl0"": ""Dr...",7.800000,350024
7692,31631257,American Reunion,2012.0,234736898.0,113.0,"{""/m/09c7w0"": ""United States of America""}","{""/m/02l7c8"": ""Romance Film"", ""/m/01z4y"": ""Com...",6.700000,215321
7693,32556787,Gone,2012.0,18100189.0,95.0,"{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller""}",7.359322,66531


## Parsing the genres and movie countries

In [27]:
rated_data['genres'] = rated_data['Movie_genres'].apply(lambda x : re.split(r':|,', x)[1::2])
rated_data['countries'] = rated_data['Movie_countries'].apply(lambda x : re.split(r':|,', x)[1::2])
rated_dat = rated_data.drop(['Movie_countries', 'Movie_genres'], axis = 1) #run once

In [28]:
rated_dat.head(3)

,Wikipedia_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,averageRating,numVotes,genres,countries
0,1762869,The Squaw Man,1914.0,244700.0,74.0,5.833333,1418,"[ ""Silent film"", ""Western"", ""Drama"", ""Indie...","[ ""United States of America""}]"
1,3333,The Birth of a Nation,1915.0,50000000.0,190.0,6.600000,46186,"[ ""Silent film"", ""Indie"", ""Costume drama"", ...","[ ""United States of America""}]"
2,4499436,Way Down East,1920.0,4500000.0,147.0,6.750000,5791,"[ ""Silent film"", ""Indie"", ""Black-and-white"",...","[ ""United States of America""}]"


In [29]:
data = pd.merge(char_no_nan, rated_dat)
data = data.drop(['Wikipedia_movie_ID'], axis=1)
data

,Actor_name,Actor_gender,Actor_date_of_birth,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,averageRating,numVotes,genres,countries
0,Dustin Farnum,M,1874-05-27,The Squaw Man,1914.0,244700.0,74.0,5.833333,1418,"[ ""Silent film"", ""Western"", ""Drama"", ""Indie...","[ ""United States of America""}]"
1,Elmer Clifton,M,1890-03-14,The Birth of a Nation,1915.0,50000000.0,190.0,6.600000,46186,"[ ""Silent film"", ""Indie"", ""Costume drama"", ...","[ ""United States of America""}]"
2,Robert Harron,M,1893-04-12,The Birth of a Nation,1915.0,50000000.0,190.0,6.600000,46186,"[ ""Silent film"", ""Indie"", ""Costume drama"", ...","[ ""United States of America""}]"
3,Walter Long,M,1879-03-05,The Birth of a Nation,1915.0,50000000.0,190.0,6.600000,46186,"[ ""Silent film"", ""Indie"", ""Costume drama"", ...","[ ""United States of America""}]"
4,Violet Wilkey,F,1903-01-10,The Birth of a Nation,1915.0,50000000.0,190.0,6.600000,46186,"[ ""Silent film"", ""Indie"", ""Costume drama"", ...","[ ""United States of America""}]"
...,...,...,...,...,...,...,...,...,...,...,...
78679,Robert Capron,M,1998-07-10,Diary of a Wimpy Kid: Dog Days,2012.0,75014170.0,94.0,6.300000,26487,"[ ""Family Film"", ""Comedy""}]","[ ""United States of America"", ""Canada""}]"
78680,Karan Brar,M,1999-01-18,Diary of a Wimpy Kid: Dog Days,2012.0,75014170.0,94.0,6.300000,26487,"[ ""Family Film"", ""Comedy""}]","[ ""United States of America"", ""Canada""}]"
78681,George Sampson,M,1993-06-29,StreetDance 2,2012.0,20788483.0,90.0,5.500000,9340,"[ ""Music"", ""Romance Film"", ""Drama""}]","[ ""United Kingdom"", ""Germany""}]"
78682,Bae Doo-na,F,1979-10-11,As One,2012.0,11794204.0,127.0,7.650000,1293,"[ ""Sports""}]","[ ""South Korea""}]"


In [30]:
# Renaming column to avoid merge issues
budget_no_nan = budget_no_nan.rename(columns={'original_title': 'Movie_name'})
budget_no_nan

,Movie_name,budget
0,Toy Story,30000000
1,Jumanji,65000000
3,Waiting to Exhale,16000000
5,Heat,60000000
6,Sabrina,58000000
...,...,...
45402,Корпоратив,2000000
45408,Марс,2000000
45409,Dikari,800000
45412,Про любоff,2000000


In [32]:
# Merging dataset with budget information and the ratings dataset
budget = pd.merge(budget_no_nan, rated_dat)
budget

,Movie_name,budget,Wikipedia_movie_ID,Movie_release_date,Movie_box_office_revenue,Movie_runtime,averageRating,numVotes,genres,countries
0,Toy Story,30000000,53085,1995.0,361958736.0,77.0,7.781818,990319,"[ ""Buddy film"", ""Adventure"", ""Children's/Fam...","[ ""United States of America""}]"
1,Jumanji,65000000,3700174,1995.0,262797249.0,104.0,6.360000,347988,"[ ""Thriller"", ""Fantasy Adventure"", ""Japanese...","[ ""United States of America"", ""Japan""}]"
2,Waiting to Exhale,16000000,972970,1995.0,81452156.0,121.0,7.600000,16224,"[ ""Ensemble Film"", ""Female buddy film"", ""Com...","[ ""United States of America""}]"
3,Heat,60000000,12555863,1986.0,2793214.0,92.0,7.202632,659457,"[ ""Thriller"", ""Crime Fiction"", ""Gangster Fil...","[ ""United States of America""}]"
4,Heat,60000000,43566,1995.0,187436818.0,170.0,7.202632,659457,"[ ""Thriller"", ""Crime Fiction"", ""Heist"", ""Ps...","[ ""United States of America""}]"
...,...,...,...,...,...,...,...,...,...,...
4011,The Assignment,5000000,9078818,1997.0,332597.0,119.0,6.710000,20083,"[ ""Thriller"", ""Period piece"", ""Drama"", ""Pol...","[ ""Canada""}]"
4012,Eating Out,50000,4447058,2004.0,155212.0,88.0,6.640000,6431,"[ ""LGBT"", ""Romantic comedy"", ""Gay"", ""Gay In...","[ ""United States of America""}]"
4013,I Was a Teenage Werewolf,82000,1180124,1957.0,2000000.0,76.0,6.350000,3117,"[ ""Science Fiction"", ""Melodrama"", ""Horror"", ...","[ ""United States of America""}]"
4014,The Magic Roundabout,20000000,2349209,2005.0,26691243.0,85.0,6.533333,474,"[ ""Computer Animation"", ""Family Film"", ""Fant...","[ ""United States of America"", ""France"", ""Uni..."


In [33]:
# Converting the budget column from string to numerical value
budget['budget'] = pd.to_numeric(budget['budget'])

In [34]:
# Creating a Profit column
budget['profit'] = budget['Movie_box_office_revenue'] - budget['budget']
budget

,Movie_name,budget,Wikipedia_movie_ID,Movie_release_date,Movie_box_office_revenue,Movie_runtime,averageRating,numVotes,genres,countries,profit
0,Toy Story,30000000,53085,1995.0,361958736.0,77.0,7.781818,990319,"[ ""Buddy film"", ""Adventure"", ""Children's/Fam...","[ ""United States of America""}]",331958736.0
1,Jumanji,65000000,3700174,1995.0,262797249.0,104.0,6.360000,347988,"[ ""Thriller"", ""Fantasy Adventure"", ""Japanese...","[ ""United States of America"", ""Japan""}]",197797249.0
2,Waiting to Exhale,16000000,972970,1995.0,81452156.0,121.0,7.600000,16224,"[ ""Ensemble Film"", ""Female buddy film"", ""Com...","[ ""United States of America""}]",65452156.0
3,Heat,60000000,12555863,1986.0,2793214.0,92.0,7.202632,659457,"[ ""Thriller"", ""Crime Fiction"", ""Gangster Fil...","[ ""United States of America""}]",-57206786.0
4,Heat,60000000,43566,1995.0,187436818.0,170.0,7.202632,659457,"[ ""Thriller"", ""Crime Fiction"", ""Heist"", ""Ps...","[ ""United States of America""}]",127436818.0
...,...,...,...,...,...,...,...,...,...,...,...
4011,The Assignment,5000000,9078818,1997.0,332597.0,119.0,6.710000,20083,"[ ""Thriller"", ""Period piece"", ""Drama"", ""Pol...","[ ""Canada""}]",-4667403.0
4012,Eating Out,50000,4447058,2004.0,155212.0,88.0,6.640000,6431,"[ ""LGBT"", ""Romantic comedy"", ""Gay"", ""Gay In...","[ ""United States of America""}]",105212.0
4013,I Was a Teenage Werewolf,82000,1180124,1957.0,2000000.0,76.0,6.350000,3117,"[ ""Science Fiction"", ""Melodrama"", ""Horror"", ...","[ ""United States of America""}]",1918000.0
4014,The Magic Roundabout,20000000,2349209,2005.0,26691243.0,85.0,6.533333,474,"[ ""Computer Animation"", ""Family Film"", ""Fant...","[ ""United States of America"", ""France"", ""Uni...",6691243.0


In [36]:
# Merging the profit dataset with character dataset
profit = pd.merge(char_no_nan, budget)
profit = profit.drop(['Wikipedia_movie_ID'], axis=1)
profit

,Actor_name,Actor_gender,Actor_date_of_birth,Movie_name,budget,Movie_release_date,Movie_box_office_revenue,Movie_runtime,averageRating,numVotes,genres,countries,profit
0,Elmer Clifton,M,1890-03-14,The Birth of a Nation,100000,1915.0,50000000.0,190.0,6.6,46186,"[ ""Silent film"", ""Indie"", ""Costume drama"", ...","[ ""United States of America""}]",49900000.0
1,Elmer Clifton,M,1890-03-14,The Birth of a Nation,8500000,1915.0,50000000.0,190.0,6.6,46186,"[ ""Silent film"", ""Indie"", ""Costume drama"", ...","[ ""United States of America""}]",41500000.0
2,Robert Harron,M,1893-04-12,The Birth of a Nation,100000,1915.0,50000000.0,190.0,6.6,46186,"[ ""Silent film"", ""Indie"", ""Costume drama"", ...","[ ""United States of America""}]",49900000.0
3,Robert Harron,M,1893-04-12,The Birth of a Nation,8500000,1915.0,50000000.0,190.0,6.6,46186,"[ ""Silent film"", ""Indie"", ""Costume drama"", ...","[ ""United States of America""}]",41500000.0
4,Walter Long,M,1879-03-05,The Birth of a Nation,100000,1915.0,50000000.0,190.0,6.6,46186,"[ ""Silent film"", ""Indie"", ""Costume drama"", ...","[ ""United States of America""}]",49900000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49053,Steve Zahn,M,1967-11-13,Diary of a Wimpy Kid: Dog Days,22000000,2012.0,75014170.0,94.0,6.3,26487,"[ ""Family Film"", ""Comedy""}]","[ ""United States of America"", ""Canada""}]",53014170.0
49054,Emily Holmes,F,1977-03-01,Diary of a Wimpy Kid: Dog Days,22000000,2012.0,75014170.0,94.0,6.3,26487,"[ ""Family Film"", ""Comedy""}]","[ ""United States of America"", ""Canada""}]",53014170.0
49055,Grayson Russell,M,1998,Diary of a Wimpy Kid: Dog Days,22000000,2012.0,75014170.0,94.0,6.3,26487,"[ ""Family Film"", ""Comedy""}]","[ ""United States of America"", ""Canada""}]",53014170.0
49056,Robert Capron,M,1998-07-10,Diary of a Wimpy Kid: Dog Days,22000000,2012.0,75014170.0,94.0,6.3,26487,"[ ""Family Film"", ""Comedy""}]","[ ""United States of America"", ""Canada""}]",53014170.0


### Adjusting for inflation

In [37]:
cf = pd.read_csv('../data/inflation_correction.csv') #inflation correction

inflation_match_profit = profit.merge(cf, how = 'inner', left_on='Movie_release_date', right_on='Year')
profit['profit_adj'] = profit['profit']/inflation_match_profit['CF']
inflation_match_data = data.merge(cf, how = 'inner', left_on='Movie_release_date', right_on='Year')
data['box_office_adj'] = data['Movie_box_office_revenue']/inflation_match_data['CF']

display(profit.sample(), data.sample())

,Actor_name,Actor_gender,Actor_date_of_birth,Movie_name,budget,Movie_release_date,Movie_box_office_revenue,Movie_runtime,averageRating,numVotes,genres,countries,profit,profit_adj
20957,Liev Schreiber,M,1967-10-04,Sphere,75000000,1998.0,37020277.0,135.0,6.65,107320,"[ ""Thriller"", ""Science Fiction"", ""Horror"", ...","[ ""United States of America""}]",-37979723.0,-5.585253e+07


,Actor_name,Actor_gender,Actor_date_of_birth,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,averageRating,numVotes,genres,countries,box_office_adj
42424,Donald Faison,M,1974-06-22,Trippin',1999.0,10017070.0,92.0,6.8,1723,"[ ""Romantic comedy"", ""Indie"", ""Comedy-drama""...","[ ""United States of America""}]",1.451749e+07


In [38]:
profit.to_csv('../data/profit_data.csv') #useful for movie regression
data.to_csv('../data/data.csv')